<a href="https://colab.research.google.com/github/cryptopatience/Main-code/blob/main/Mag_7_darkpool_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치 및 임포트
# !pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== 설정 및 종목 리스트 ====================
MAG7_STOCKS = {
    'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Alphabet',
    'AMZN': 'Amazon', 'NVDA': 'NVIDIA', 'META': 'Meta',
    'TSLA': 'Tesla', 'COIN': 'Coinbase', 'IBIT': 'Bitcoin ETF'
}

# ==================== 데이터 수집 함수 ====================

def get_market_volume(ticker, days_back=65):
    """Yahoo Finance에서 전체 시장 거래량 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back + 10)
        df = stock.history(start=start_date, end=end_date)
        return df['Volume']
    except:
        return None

def get_yf_short_info(ticker):
    """Yahoo Finance에서 공매도 정보 가져오기 (표준 지표)"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        return {
            'shares_short': info.get('sharesShort', 0),  # 공매도 잔고
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100,  # Short % of Float
            'short_ratio_days': info.get('shortRatio', 0),  # Days to Cover ⭐
            'shares_outstanding': info.get('sharesOutstanding', 0)
        }
    except:
        return {
            'shares_short': 0,
            'short_percent_float': 0,
            'short_ratio_days': 0,
            'shares_outstanding': 0
        }

def get_finra_data_full(ticker, days_back=60):
    """
    FINRA 데이터 수집 및 핵심 지표 계산 (60일 history 포함)

    지표:
    1. dp_ratio: Dark Pool 비중 (FINRA 전체 / 시장 전체)
    2. dp_short_ratio: DP 내부 공매도 비율 (FINRA 공매도 / FINRA 전체)
    3. dp_short_market_impact: DP 공매도의 시장 영향 (FINRA 공매도 / 시장 전체)
    """
    try:
        today = datetime.now()
        data_list = []

        # Yahoo Finance 공매도 정보
        yf_short_info = get_yf_short_info(ticker)
        yf_shares_short = yf_short_info['shares_short']

        # Yahoo 전체 거래량
        market_volumes = get_market_volume(ticker, days_back)
        if market_volumes is None or market_volumes.empty:
            return None

        # FINRA 데이터 루프 (최근 N일)
        for days in range(days_back + 5):
            check_date = today - timedelta(days=days)
            if check_date.weekday() >= 5:
                continue

            date_str = check_date.strftime('%Y%m%d')
            date_key = check_date.strftime('%Y-%m-%d')

            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=3)
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text), sep='|')
                    if 'Symbol' in df.columns:
                        df.rename(columns={'Symbol': 'symbol'}, inplace=True)
                    if 'ShortVolume' in df.columns:
                        df.rename(columns={'ShortVolume': 'shortVolume'}, inplace=True)
                    if 'TotalVolume' in df.columns:
                        df.rename(columns={'TotalVolume': 'totalVolume'}, inplace=True)

                    row = df[df['symbol'] == ticker.upper()]

                    if not row.empty:
                        finra_total = int(row.iloc[0]['totalVolume'])
                        finra_short = int(row.iloc[0]['shortVolume'])

                        # Yahoo Volume 매칭
                        market_vol = 0
                        if date_key in market_volumes.index:
                            market_vol = market_volumes.loc[date_key]
                        else:
                            for idx in market_volumes.index:
                                if idx.strftime('%Y-%m-%d') == date_key:
                                    market_vol = market_volumes[idx]
                                    break

                        if market_vol > 0:
                            # 지표 계산
                            dp_ratio = (finra_total / market_vol) * 100
                            dp_short_ratio = (finra_short / finra_total) * 100
                            dp_short_market_impact = (finra_short / market_vol) * 100

                            if dp_ratio > 100:
                                dp_ratio = 100

                            data_list.append({
                                'date': date_key,
                                'dp_ratio': round(dp_ratio, 2),
                                'dp_short_ratio': round(dp_short_ratio, 2),
                                'dp_short_market_impact': round(dp_short_market_impact, 2),
                                'market_vol': market_vol,
                                'yf_shares_short': yf_shares_short
                            })

                if len(data_list) >= days_back:
                    break
            except:
                continue

        if not data_list:
            return None

        df_hist = pd.DataFrame(data_list).sort_values('date')
        df_hist['dp_short_ratio_10d_avg'] = df_hist['dp_short_ratio'].rolling(window=10, min_periods=1).mean()
        latest = df_hist.iloc[-1]
        recent_10d_avg = df_hist.iloc[-10:]['dp_short_ratio'].mean()
        dp_short_change = latest['dp_short_ratio'] - recent_10d_avg

        return {
            'ticker': ticker,
            'name': MAG7_STOCKS[ticker],
            'latest_date': latest['date'],
            'dp_ratio': latest['dp_ratio'],
            'dp_short_ratio': latest['dp_short_ratio'],
            'dp_short_market_impact': latest['dp_short_market_impact'],
            'dp_short_10d_avg': latest['dp_short_ratio_10d_avg'],
            'dp_short_change_pct': dp_short_change,
            'yf_shares_short': latest['yf_shares_short'],
            'yf_short_percent_float': yf_short_info['short_percent_float'],
            'yf_short_ratio_days': yf_short_info['short_ratio_days'],  # ⭐ Days to Cover
            'history': df_hist
        }

    except Exception as e:
        return None

# ==================== 메인 분석 실행 ====================

print("="*100)
print("🚀 MAG 7+2: Dark Pool & Short Interest 심층 분석 (표준 지표 적용)")
print("="*100)

analysis_results = []
print("\n데이터 수집 중...", end=" ")

for ticker in MAG7_STOCKS.keys():
    print(f"{ticker}..", end=" ")
    res = get_finra_data_full(ticker, days_back=60)
    if res:
        analysis_results.append(res)

print("\n완료!")

if not analysis_results:
    print("❌ 데이터를 가져올 수 없습니다.")
else:
    # Calculate finra_yf_short_ratio for each item in analysis_results before creating df_main
    for item in analysis_results:
        yf_shares_short = item['yf_shares_short']
        if yf_shares_short > 0 and not item['history'].empty:
            # Reconstruct daily FINRA short volume from dp_short_market_impact and latest market_vol
            latest_market_vol = item['history'].iloc[-1]['market_vol']
            daily_finra_short_vol = (item['dp_short_market_impact'] * latest_market_vol) / 100

            finra_yf_short_ratio = (daily_finra_short_vol / yf_shares_short) * 100
            item['finra_yf_short_ratio'] = finra_yf_short_ratio
        else:
            item['finra_yf_short_ratio'] = 0.0 # Assign 0 if yf_shares_short is 0 or history is empty

    df_main = pd.DataFrame([{k:v for k,v in r.items() if k != 'history'} for r in analysis_results])

    # 시그널 컬럼 생성
    def create_signal(row):
        # 1. Short Squeeze 임박 (Days to Cover 높음 + DP Short 급락)
        if row['yf_short_ratio_days'] > 5 and row['dp_short_change_pct'] < -5:
            return '🔥 Short Squeeze 임박!'

        # 2. 공매도 청산 신호
        if row['dp_short_change_pct'] < -5:
            return '🟢 급락 (청산 신호)'

        # 3. Days to Cover 매우 높음
        if row['yf_short_ratio_days'] > 7:
            return '🔴🔴 극심한 공매도 (7일+)'

        # 4. 기관 강한 약세
        if row['dp_ratio'] > 50 and row['dp_short_ratio'] > 55:
            return '🔴 기관 강한 약세'

        # 5. 기관 매집
        if row['dp_ratio'] > 50 and row['dp_short_ratio'] < 45:
            return '💚 기관 매집 가능성'

        # 6. Days to Cover 정상
        if row['yf_short_ratio_days'] < 3:
            return '✅ 건강 (DTC <3일)'

        return '⚪ 관망/정상'

    df_main['Signal'] = df_main.apply(create_signal, axis=1)
    df_main = df_main.sort_values('yf_short_ratio_days', ascending=False)

    # ==================== 핵심 지표 해석 가이드 ====================

    print("\n" + "="*120)
    print("📚 핵심 지표 체계 및 상세 해석 가이드")
    print("="*120)
    print("""
    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 표준 공매도 지표 (Yahoo Finance 제공) - 가장 중요!
       Short Float = (Number of Shorted Shares / Shares Float) × 100%
       Short Ratio = Number of Shorted Shares / Average Daily Trading Volume                                    │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 1️⃣ Short Ratio (Days to Cover) ⭐⭐⭐ 최우선 핵심 지표!                                              │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: 공매도 잔고 / 평균 일일 거래량                                                      │
    │                                                                                                │
    │ 💡 의미:                                                                                       │
    │    • 현재 쌓여있는 모든 공매도 포지션을 청산하는데 며칠 걸리는가?                             │
    │    • 공매도 세력이 "탈출"하는데 필요한 시간                                                   │
    │    • Short Squeeze 위험도의 직접적 측정치                                                     │
    │                                                                                                │
    │ 📊 해석 기준:                                                                                  │
    │    • <3일 (초록) : 정상 - 공매도 청산 용이, 위험 낮음                                         │
    │    • 3-5일 (주황): 주의 - 공매도 압력 존재, 변동성 가능                                       │
    │    • 5-7일 (빨강): 높음 - Short Squeeze 가능성, 급등 잠재력                                   │
    │    • >7일 (적색) : 매우 높음 - Short Squeeze 고위험, 폭발적 상승 가능                         │
    │                                                                                                │
    │ 🎯 실전 활용:                                                                                  │
    │    • Days to Cover >5일 + 호재 발생 = 🔥 Short Squeeze 폭발!                                 │
    │    • Days to Cover >7일 + DP Short 급락(-5%p) = 🚀 청산 시작, 강력 매수 신호               │
    │    • Days to Cover <3일 = 공매도 압력 적음, 안정적 거래 가능                                  │
    │                                                                                                │
    │ ⚠️ 주의사항:                                                                                   │
    │    • 높다고 무조건 좋은 것은 아님! 공매도 세력이 맞을 수도 있음                               │
    │    • 하지만 일단 상승 시작하면 → 청산 압력으로 폭등 가능                                      │
    │    • GameStop(GME) 사태가 대표적 예시 (Days to Cover 매우 높았음)                            │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 2️⃣ Short % of Float (Short Float) ⭐⭐                                                         │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (공매도 잔고 / 유통주식수) × 100                                                    │
    │                                                                                                │
    │ 💡 의미:                                                                                       │
    │    • 시장에서 거래 가능한 주식(Float) 중 몇 %가 공매도되었는가?                               │
    │    • 시장 참여자들의 약세 심리 강도                                                           │
    │    • 유통주식 대비 공매도 비중                                                                │
    │                                                                                                │
    │ 📊 해석 기준:                                                                                  │
    │    • <2% (초록)  : 매우 낮음 - 시장의 강한 신뢰                                               │
    │    • 2-5% (연두) : 정상 - 일반적인 수준                                                       │
    │    • 5-10% (주황): 주의 - 공매도 세력의 관심 증가                                             │
    │    • >10% (빨강) : 높음 - 강한 약세 베팅 + Short Squeeze 잠재력                              │
    │    • >20% (적색) : 매우 높음 - 극단적 약세 베팅                                               │
    │                                                                                                │
    │ 🎯 실전 활용:                                                                                  │
    │    • Float >10% + Days to Cover >5일 = 💣 최고 위험도 (or 기회)                             │
    │    • Float <5% = 건강한 종목, 공매도 압력 낮음                                                │
    │                                                                                                │
    │ ⚠️ Days to Cover와의 차이:                                                                    │
    │    • Short Float: 유통주식 대비 비율 (정적 지표, 장기 관점)                                  │
    │    • Days to Cover: 거래량 대비 청산 일수 (동적 지표, 위험도 측정)                           │
    │    • 두 지표 모두 높으면 → 극도로 위험한(또는 기회가 되는) 상황!                             │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 FINRA 장외 거래 지표 (Dark Pool Analysis)                                                   │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 3️⃣ DP Ratio (Dark Pool 비중) ⭐                                                                │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 전체 거래량 / 전체 시장 거래량) × 100                                        │
    │ 💡 의미: 전체 시장에서 장외(비공개) 거래가 차지하는 비중                                       │
    │ 📊 해석: >50% = 기관 과열, 40-50% = 기관 관심, <40% = 정상                                   │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 4️⃣ DP Internal Short Ratio (DP 내부 공매도) ⭐                                                │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 공매도량 / FINRA 전체 거래량) × 100                                          │
    │ 💡 의미: 장외 거래 중 공매도가 차지하는 비율 (기관의 숨겨진 심리)                              │
    │ 📊 해석: >55% = 강한 약세, 45-55% = 중립, <45% = 강세                                         │
    │ 🎯 활용: DP Ratio와 함께 봐야 정확! DP↑ + DP Short↓ = 매집, DP↑ + DP Short↑ = 분산        │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 5️⃣ DP Short Market Impact (DP 공매도 시장 영향)                                               │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 공매도량 / 전체 시장 거래량) × 100                                           │
    │ 💡 의미: 전체 시장 거래량 대비 장외 공매도의 실제 영향력                                       │
    │ 📊 해석: 장외 공매도가 전체 시장에 미치는 절대적 규모                                          │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 6️⃣ FINRA/YF Short Ratio (공매도 신선도) - 우리의 독자 지표                                    │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 일일 공매도량 / YF 전체 공매도 잔고) × 100                                   │
    │ 💡 의미: 전체 공매도 중 오늘 장외에서 발생한 비율 (신규 vs 기존)                              │
    │ 📊 해석: >50% = 활발한 신규 공매도, 10-50% = 정상, <10% = 청산 진행 중                       │
    │ 🎯 활용: Days to Cover >5일 + FINRA/YF <10% = 🟢 청산 시작! (매수 기회)                     │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘
    """)
    print("="*120)

    # ==================== 종합 시나리오 분석 ====================

    print("\n" + "="*120)
    print("🎯 종합 시나리오 분석 - 지표 조합으로 상황 파악하기")
    print("="*120)
    print("""
    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 🔥 시나리오 1: Short Squeeze 임박! (최고 기회 or 최고 위험)                                    │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover > 5일 (공매도 청산 어려움)                                                    │
    │  • Short % Float > 10% (높은 공매도 비율)                                                      │
    │  • DP Short 급락 (10일 대비 -5%p 이상)                                                         │
    │  • FINRA/YF < 10% (신규 공매도 없음, 청산 시작)                                                │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 세력이 쌓아놓은 포지션은 많은데(Float >10%, DTC >5일),                                 │
    │  청산하기 시작했고(DP Short 급락, FINRA/YF 낮음),                                              │
    │  청산에 시간도 오래 걸림(DTC >5일)                                                             │
    │  → 💥 연쇄 청산으로 폭등 가능성!                                                              │
    │                                                                                                │
    │ 🎯 전략: 적극적 매수 진입 (단, 리스크 높음!)                                                   │
    │                                                                                                │
    │ 📌 역사적 사례: GameStop (GME), AMC 등                                                         │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 🔴 시나리오 2: 공매도 공격 진행 중                                                             │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover 증가 추세                                                                     │
    │  • DP Short > 55% (장외에서 강한 공매도)                                                       │
    │  • DP Short 급등 (10일 대비 +5%p 이상)                                                         │
    │  • FINRA/YF > 50% (활발한 신규 공매도)                                                         │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 세력이 적극적으로 포지션을 늘리고 있음                                                 │
    │  → 하락 압력 증가 예상                                                                        │
    │                                                                                                │
    │ 🎯 전략: 관망 또는 단기 숏 포지션 (역추세 매수는 위험)                                         │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 💚 시나리오 3: 기관 매집 (장외 매수)                                                           │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • DP Ratio > 50% (기관 개입 강함)                                                             │
    │  • DP Short < 45% (장외에서 매수 우위)                                                         │
    │  • Days to Cover < 3일 (공매도 압력 낮음)                                                      │
    │  • Short % Float < 5% (건강한 수준)                                                            │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  기관들이 장외에서 조용히 매수 중, 공매도 압력도 낮음                                          │
    │  → 안정적 상승 가능성                                                                         │
    │                                                                                                │
    │ 🎯 전략: 안정적 매수 또는 홀딩                                                                 │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ ✅ 시나리오 4: 건강한 종목 (이상적)                                                            │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover < 3일                                                                         │
    │  • Short % Float < 5%                                                                          │
    │  • DP Short 40-50% (중립)                                                                      │
    │  • FINRA/YF 10-50% (정상 회전)                                                                 │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 압력 낮고, 기관 활동도 정상 범위                                                       │
    │  → 안정적 거래 가능, 변동성 낮음                                                              │
    │                                                                                                │
    │ 🎯 전략: 펀더멘털 분석 기반 투자                                                               │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ ⚠️ 시나리오 5: 주의 필요 (공매도 많지만 청산은 안됨)                                           │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover > 7일 (매우 높음)                                                             │
    │  • Short % Float > 15%                                                                         │
    │  • BUT, DP Short 변화 없음 (횡보)                                                              │
    │  • FINRA/YF 10-50% (정상 회전)                                                                 │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 잔고는 많지만 청산도, 추가 공격도 없는 교착 상태                                       │
    │  → 촉매(호재/악재) 기다리는 중, 어느 방향으로든 큰 움직임 가능                                 │
    │                                                                                                │
    │ 🎯 전략: 대기, 촉매 확인 후 대응                                                               │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘
    """)
    print("="*120)

    # ==================== 통합 상세 비교 테이블 ====================

    print("\n" + "="*150)
    print("📋 통합 상세 비교 테이블: 표준 지표 중심 분석")
    print("="*150)
    print("""
    💡 테이블 읽는 법:
    • Days_to_Cover >5일 = Short Squeeze 위험 구간
    • Short_%_Float >10% = 높은 공매도 비율
    • DP내부공매도 >55% = 장외에서 강한 약세
    • 1일vs10일 <-5%p = 청산 시작 신호
    • FINRA/YF >50% = 활발한 신규 공매도, <10% = 청산 진행
    """)

    df_display = df_main.copy()
    df_display = df_display.rename(columns={
        'ticker': '티커',
        'name': '종목명',
        'yf_short_ratio_days': 'Days_to_Cover',
        'yf_short_percent_float': 'Short_%_Float',
        'dp_ratio': 'DP비중_%',
        'dp_short_ratio': 'DP내부공매도_%',
        'dp_short_10d_avg': 'DP_10일평균',
        'dp_short_change_pct': '1일vs10일',
        'dp_short_market_impact': 'DP→시장_%',
        'finra_yf_short_ratio': 'FINRA/YF_%',
        'Signal': '신호'
    })

    table_cols = ['티커', '종목명', 'Days_to_Cover', 'Short_%_Float',
                  'DP비중_%', 'DP내부공매도_%', 'DP_10일평균', '1일vs10일',
                  'DP→시장_%', 'FINRA/YF_%', '신호']

    # 포맷팅
    df_display['Days_to_Cover'] = df_display['Days_to_Cover'].apply(lambda x: f"{x:.2f}일")
    df_display['Short_%_Float'] = df_display['Short_%_Float'].apply(lambda x: f"{x:.2f}%")
    df_display['DP비중_%'] = df_display['DP비중_%'].apply(lambda x: f"{x:.2f}%")
    df_display['DP내부공매도_%'] = df_display['DP내부공매도_%'].apply(lambda x: f"{x:.2f}%")
    df_display['DP_10일평균'] = df_display['DP_10일평균'].apply(lambda x: f"{x:.2f}%")
    df_display['1일vs10일'] = df_display['1일vs10일'].apply(lambda x: f"{x:+.2f}%p")
    df_display['DP→시장_%'] = df_display['DP→시장_%'].apply(lambda x: f"{x:.2f}%")
    df_display['FINRA/YF_%'] = df_display['FINRA/YF_%'].apply(lambda x: f"{x:.1f}%")

    print(df_display[table_cols].to_string(index=False))
    print("="*150)

    # ==================== 요약 통계 ====================

    print("\n" + "="*100)
    print("📊 전체 시장 개요")
    print("="*100)
    print(f"평균 Days to Cover:    {df_main['yf_short_ratio_days'].mean():.2f}일")
    print(f"평균 Short % Float:    {df_main['yf_short_percent_float'].mean():.2f}%")
    print(f"평균 DP 비중:          {df_main['dp_ratio'].mean():.2f}%")
    print(f"평균 DP 내부 공매도:   {df_main['dp_short_ratio'].mean():.2f}%")
    print()
    print(f"Short Squeeze 위험 (DTC >5일):        {len(df_main[df_main['yf_short_ratio_days'] > 5])}개")
    print(f"높은 공매도 비율 (Float >10%):        {len(df_main[df_main['yf_short_percent_float'] > 10])}개")
    print(f"청산 신호 (1일vs10일 <-5%p):           {len(df_main[df_main['dp_short_change_pct'] < -5])}개")
    print(f"활발한 신규 공매도 (FINRA/YF >50%):    {len(df_main[df_main['finra_yf_short_ratio'] > 50])}개")
    print("="*100)

    # ==================== 차트 1: Days to Cover (가장 중요!) ====================

    print("\n" + "="*120)
    print("📊 Chart 1: Days to Cover - Short Squeeze 위험도 측정 ⭐⭐⭐ 최우선 지표!")
    print("="*120)
    print("""
    💡 Days to Cover (청산 소요 일수) 상세 해석:

    🔍 이 지표가 가장 중요한 이유:
    • Short Squeeze 가능성을 직접적으로 측정
    • 공매도 세력의 "탈출 난이도" 표시
    • 변동성 및 급등 가능성 예측

    📊 해석 기준 (매우 중요!):
    • <3일 (초록): 정상
      → 공매도 세력이 빠르게 청산 가능
      → 변동성 낮음, 안정적 거래
      → 공매도 압력 미미

    • 3-5일 (노랑): 주의
      → 공매도 청산에 시간 소요
      → 호재 발생 시 변동성 증가 가능
      → 모니터링 필요

    • 5-7일 (주황): 위험
      → Short Squeeze 가능성 있음
      → 공매도 세력 탈출 어려움
      → 급등 잠재력 존재

    • >7일 (빨강): 극도 위험!
      → Short Squeeze 고위험 구간
      → 한 번 상승 시작하면 연쇄 청산으로 폭등 가능
      → GameStop(GME) 사태가 대표적 예시 (Days to Cover 매우 높았음)

    🎯 실전 활용 전략:
    1. DTC >7일 + 호재 발생 = 🚀 폭발적 상승 가능
    2. DTC >5일 + DP Short 급락 = 🔥 청산 시작, 매수 기회
    3. DTC <3일 = 안정적 종목, 펀더멘털 위주 투자

    ⚠️ 함정 주의:
    • DTC가 높다고 무조건 오르는 것은 아님!
    • 공매도 세력이 맞을 수도 있음 (주가 계속 하락 가능)
    • 하지만 일단 반등 시작하면 → 청산 압력으로 급등!
    """)

    fig1 = go.Figure()

    # Days to Cover 기준 색상
    colors_dtc = []
    for x in df_main['yf_short_ratio_days']:
        if x < 3:
            colors_dtc.append('green')
        elif x < 5:
            colors_dtc.append('yellow')
        elif x < 7:
            colors_dtc.append('orange')
        else:
            colors_dtc.append('red')

    fig1.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['yf_short_ratio_days'],
        text=[f"{x:.2f}일" for x in df_main['yf_short_ratio_days']],
        textposition='auto',
        marker_color=colors_dtc,
        hovertemplate='<b>%{x}</b><br>Days to Cover: %{y:.2f}일<br>청산 소요 기간<extra></extra>'
    ))

    fig1.add_hline(y=3, line_dash="dash", line_color="green", annotation_text="정상 (3일)")
    fig1.add_hline(y=5, line_dash="dash", line_color="orange", annotation_text="주의 (5일)")
    fig1.add_hline(y=7, line_dash="dash", line_color="red", annotation_text="위험 (7일)")

    fig1.update_layout(
        title='Days to Cover (Short Ratio): 공매도 청산 소요 일수 - Short Squeeze 핵심 지표',
        height=550,
        template='plotly_white',
        xaxis_title='종목',
        yaxis_title='Days to Cover (일)'
    )
    fig1.show()

    # ==================== 차트 2: Short % of Float ====================

    print("\n" + "="*120)
    print("📊 Chart 2: Short % of Float - 유통주식 대비 공매도 비율")
    print("="*120)
    print("""
    💡 Short % of Float 해석:

    🔍 의미:
    • 시장에서 거래 가능한 주식 중 몇 %가 공매도되었는가?
    • 시장 참여자들의 약세 심리 강도 측정

    📊 해석 기준:
    • <2%: 매우 낮음 (시장의 강한 신뢰)
    • 2-5%: 정상 범위 (건강한 시장)
    • 5-10%: 주의 (공매도 세력 관심 증가)
    • >10%: 높음 (강한 약세 베팅, Squeeze 잠재력도 높음)

    🎯 Days to Cover와 함께 보기:
    • Float >10% + DTC >5일 = 💥 극도로 위험(또는 기회!)
    • Float <5% + DTC <3일 = ✅ 안정적 종목
    """)

    fig2 = go.Figure()

    colors_float = ['green' if x < 2 else 'yellowgreen' if x < 5 else 'orange' if x < 10 else 'red'
                   for x in df_main['yf_short_percent_float']]

    fig2.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['yf_short_percent_float'],
        text=[f"{x:.2f}%" for x in df_main['yf_short_percent_float']],
        textposition='auto',
        marker_color=colors_float,
        hovertemplate='<b>%{x}</b><br>Short % Float: %{y:.2f}%<extra></extra>'
    ))

    fig2.add_hline(y=2, line_dash="dash", line_color="green", annotation_text="매우 낮음 (2%)")
    fig2.add_hline(y=5, line_dash="dash", line_color="yellowgreen", annotation_text="정상 (5%)")
    fig2.add_hline(y=10, line_dash="dash", line_color="red", annotation_text="높음 (10%)")

    fig2.update_layout(
        title='Short % of Float: 유통주식 대비 공매도 비율',
        xaxis_title='종목',
        yaxis_title='Short % of Float (%)',
        height=550,
        template='plotly_white'
    )
    fig2.show()

    # ==================== 차트 3: 공매도 종합 비교 ====================

    print("\n" + "="*120)
    print("📊 Chart 3: 공매도 종합 비교 - Dark Pool vs 전체 시장")
    print("="*120)
    print("""
    💡 4가지 지표 종합 비교:

    1️⃣ DP Internal Short (파란색)
       • 장외 거래 중 공매도 비율
       • 기관의 은밀한 심리

    2️⃣ DP Market Impact (회색)
       • 전체 시장 대비 장외 공매도 영향
       • 절대적 규모

    3️⃣ FINRA/YF Ratio (보라색) ⭐
       • 오늘 발생한 공매도 / 전체 잔고
       • 공매도 "신선도" 측정
       • >50% = 신규 공매도 활발
       • <10% = 청산 진행

    📈 종합 해석:
    • DP Internal 높음 + FINRA/YF 높음 = 신규 공매도 공격
    • DP Internal 높음 + FINRA/YF 낮음 = 청산 시작 (기회!)
    """)

    fig3 = go.Figure()

    # DP 내부 공매도
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['dp_short_ratio'],
        name='DP Internal Short',
        marker_color='darkblue',
        text=df_main['dp_short_ratio'].round(1),
        textposition='auto'
    ))

    # DP→시장 공매도
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['dp_short_market_impact'],
        name='DP Market Impact',
        marker_color='gray',
        text=df_main['dp_short_market_impact'].round(1),
        textposition='auto'
    ))

    # FINRA/YF 비율
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['finra_yf_short_ratio'],
        name='FINRA/YF Ratio (신선도)',
        marker_color='purple',
        text=df_main['finra_yf_short_ratio'].round(1),
        textposition='auto'
    ))

    fig3.add_hline(y=50, line_dash="dash", line_color="orange", annotation_text="50% 기준")

    fig3.update_layout(
        title='공매도 종합 비교: DP Internal vs Market Impact vs 신선도',
        barmode='group',
        height=550,
        template='plotly_white',
        xaxis_title='종목',
        yaxis_title='비율 (%)',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig3.show()

    # ==================== 차트 4: 4분면 분석 ====================

    print("\n" + "="*120)
    print("📊 Chart 4: Days to Cover vs Short % Float - 위험도 매트릭스")
    print("="*120)
    print("""
    💡 2D 위험도 분석:

    X축: Short % of Float (공매도 비율)
    Y축: Days to Cover (청산 소요 일수)

    🔥 우상단 (빨강): 극도 위험!
       • Float 높음 + DTC 높음
       • Short Squeeze 폭발 가능 구간
       • 가장 변동성 높은 영역

    💚 좌하단 (초록): 안정적
       • Float 낮음 + DTC 낮음
       • 공매도 압력 거의 없음
       • 펀더멘털 투자 적합

    버블 크기 = DP 내부 공매도 비율
    버블 색상 = Days to Cover (빨간색일수록 위험)
    """)

    fig4 = go.Figure()

    fig4.add_trace(go.Scatter(
        x=df_main['yf_short_percent_float'],
        y=df_main['yf_short_ratio_days'],
        mode='markers+text',
        text=df_main['ticker'],
        textposition='top center',
        marker=dict(
            size=df_main['dp_short_ratio'] * 1.2,
            color=df_main['yf_short_ratio_days'],
            colorscale='RdYlGn_r',
            showscale=True,
            colorbar=dict(title="Days to Cover"),
            line=dict(width=1, color='black')
        ),
        hovertemplate='<b>%{text}</b><br>Float: %{x:.2f}%<br>DTC: %{y:.2f}일<extra></extra>'
    ))

    fig4.add_vline(x=10, line_dash="dot", line_color="gray", line_width=2)
    fig4.add_hline(y=5, line_dash="dot", line_color="gray", line_width=2)

    fig4.add_annotation(x=15, y=8, text="<b>🔥 극도 위험<br>Squeeze Zone</b>",
                      showarrow=False, font=dict(color="darkred", size=12),
                      bgcolor="rgba(255,200,200,0.3)", bordercolor="red", borderwidth=2, borderpad=4)
    fig4.add_annotation(x=3, y=2, text="<b>💚 안정적<br>Safe Zone</b>",
                      showarrow=False, font=dict(color="darkgreen", size=12),
                      bgcolor="rgba(200,255,200,0.3)", bordercolor="green", borderwidth=2, borderpad=4)

    fig4.update_layout(
        title='Short Squeeze Risk Matrix: Float % vs Days to Cover',
        xaxis_title='Short % of Float (%)',
        yaxis_title='Days to Cover (일)',
        height=600,
        template='plotly_white'
    )
    fig4.show()

    # ==================== 차트 5 & 6: 시계열 분석 ====================

    print("\n" + "="*120)
    print("📊 Chart 5 & 6: 전체 종목 시계열 분석 - 60일 트렌드")
    print("="*120)
    print("""
    💡 시계열 차트 해석:

    1️⃣ 상단: DP 비중 Trend
       • 기관 개입 강도 변화
       • 상승 = 기관 활동 증가

    2️⃣ 중단: DP 내부 공매도 Trend + 10일 평균
       • 장외 공매도 심리 변화
       • 10일 평균 대비 ±5%p = 중요한 변곡점
       • 급락(-5%p) = 🟢 청산 신호
       • 급등(+5%p) = 🔴 공매도 공격

    📈 패턴 인식:
    • 중단 급락 = 공매도 청산 시작 (매수 기회)
    • 중단 급등 = 공매도 공격 시작 (주의)
    • 상단↑ + 중단↓ = 기관 매집
    • 상단↑ + 중단↑ = 기관 분산
    """)

    for item in analysis_results:
        ticker = item['ticker']
        name = item['name']
        df_hist = item['history']

        print(f"\n🔍 {ticker} ({name}) - DTC: {item['yf_short_ratio_days']:.2f}일, Float: {item['yf_short_percent_float']:.2f}%")

        fig_ts = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            subplot_titles=(
                f"{ticker} - Dark Pool 비중 Trend",
                f"{ticker} - DP 내부 공매도 Trend + 10일 평균"
            ),
            vertical_spacing=0.12
        )

        # 차트 1: DP 비중
        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_ratio'],
            mode='lines+markers',
            name='DP 비중',
            line=dict(color='blue', width=2),
            showlegend=False,
            hovertemplate='날짜: %{x}<br>DP비중: %{y:.2f}%<extra></extra>'
        ), row=1, col=1)

        fig_ts.add_hline(y=50, line_dash="dot", line_color="red",
                        annotation_text="과열 (50%)", row=1, col=1)

        # 차트 2: DP 내부 공매도 + 10일 평균
        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_short_ratio'],
            mode='lines+markers',
            name='DP 내부 공매도',
            line=dict(color='orange', width=2),
            showlegend=False,
            hovertemplate='DP Short: %{y:.2f}%<extra></extra>'
        ), row=2, col=1)

        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_short_ratio_10d_avg'],
            mode='lines',
            name='10일 평균',
            line=dict(color='gray', dash='dot', width=1.5),
            showlegend=False,
            hovertemplate='10일평균: %{y:.2f}%<extra></extra>'
        ), row=2, col=1)

        fig_ts.add_hline(y=50, line_dash="dot", line_color="gray",
                        annotation_text="분기점", row=2, col=1)

        # 급락/급등 구간 하이라이트
        for i in range(1, len(df_hist)):
            prev = df_hist.iloc[i-1]
            curr = df_hist.iloc[i]
            change = curr['dp_short_ratio'] - prev['dp_short_ratio']

            if change < -5:  # 급락 (녹색)
                fig_ts.add_vrect(
                    x0=prev['date'], x1=curr['date'],
                    fillcolor="green", opacity=0.15,
                    layer="below", line_width=0,
                    row=2, col=1
                )
            elif change > 5:  # 급등 (빨간색)
                fig_ts.add_vrect(
                    x0=prev['date'], x1=curr['date'],
                    fillcolor="red", opacity=0.15,
                    layer="below", line_width=0,
                    row=2, col=1
                )

        fig_ts.update_layout(
            height=700,
            title_text=f"📊 {ticker} ({name}) - 60일 트렌드 | DTC: {item['yf_short_ratio_days']:.2f}일",
            template='plotly_white',
            hovermode='x unified'
        )

        fig_ts.update_xaxes(title_text="날짜", row=2, col=1)
        fig_ts.update_yaxes(title_text="DP 비중 (%)", row=1, col=1)
        fig_ts.update_yaxes(title_text="DP 내부 공매도 (%)", row=2, col=1)

        fig_ts.show()

    # ==================== 최종 요약 및 인사이트 ====================

    print("\n" + "="*120)
    print("✨ 최종 분석 요약 - 핵심 인사이트")
    print("="*120)

    # 1. Short Squeeze 고위험 종목
    squeeze_risk = df_main[(df_main['yf_short_ratio_days'] > 5) & (df_main['yf_short_percent_float'] > 10)]
    if not squeeze_risk.empty:
        print(f"\n🔥 Short Squeeze 고위험 종목 (DTC >5일 & Float >10%):")
        for _, row in squeeze_risk.iterrows():
            dtc_change = "청산 중" if row['dp_short_change_pct'] < -5 else "유지" if abs(row['dp_short_change_pct']) < 5 else "증가 중"
            print(f"   • {row['ticker']}: DTC {row['yf_short_ratio_days']:.2f}일, Float {row['yf_short_percent_float']:.2f}% - {dtc_change}")

    # 2. 청산 진행 중
    squeeze_starting = df_main[(df_main['yf_short_ratio_days'] > 5) & (df_main['dp_short_change_pct'] < -5)]
    if not squeeze_starting.empty:
        print(f"\n🟢 청산 시작 종목 (DTC >5일 & DP Short 급락):")
        for _, row in squeeze_starting.iterrows():
            finra_status = "청산 진행" if row['finra_yf_short_ratio'] < 10 else "정상"
            print(f"   • {row['ticker']}: {row['dp_short_change_pct']:+.2f}%p 급락, FINRA/YF {row['finra_yf_short_ratio']:.1f}% ({finra_status})")

    # 3. 신규 공매도 공격
    new_short_attack = df_main[(df_main['dp_short_change_pct'] > 5) & (df_main['finra_yf_short_ratio'] > 50)]
    if not new_short_attack.empty:
        print(f"\n🔴 신규 공매도 공격 진행 (DP Short 급등 & FINRA/YF >50%):")
        for _, row in new_short_attack.iterrows():
            print(f"   • {row['ticker']}: {row['dp_short_change_pct']:+.2f}%p 급등, FINRA/YF {row['finra_yf_short_ratio']:.1f}%")

    # 4. 건강한 종목
    healthy = df_main[(df_main['yf_short_ratio_days'] < 3) & (df_main['yf_short_percent_float'] < 5)]
    if not healthy.empty:
        print(f"\n✅ 건강한 종목 (DTC <3일 & Float <5%):")
        for _, row in healthy.iterrows():
            print(f"   • {row['ticker']}: DTC {row['yf_short_ratio_days']:.2f}일, Float {row['yf_short_percent_float']:.2f}%")

    # 5. 기관 매집 의심
    accumulation = df_main[(df_main['dp_ratio'] > 50) & (df_main['dp_short_ratio'] < 45)]
    if not accumulation.empty:
        print(f"\n💚 기관 매집 가능성 (DP >50% & DP Short <45%):")
        for _, row in accumulation.iterrows():
            print(f"   • {row['ticker']}: DP {row['dp_ratio']:.1f}%, DP Short {row['dp_short_ratio']:.1f}%")

    print("\n" + "="*120)
    print("📊 데이터 출처:")
    print("   • Yahoo Finance: Days to Cover, Short % of Float (표준 지표)")
    print("   • FINRA: Dark Pool 거래 데이터 (장외 거래 분석)")
    print(f"🕐 분석 완료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*120)

    print("\n💡 투자 시 주의사항:")
    print("""
    1. Days to Cover가 높다고 무조건 오르는 것은 아닙니다.
    2. 공매도 세력이 맞을 수도 있으며, 주가는 계속 하락할 수 있습니다.
    3. 하지만 일단 반등이 시작되면, DTC가 높을수록 Short Squeeze로 폭등할 가능성이 큽니다.
    4. 이 분석은 기술적 지표일 뿐, 펀더멘털 분석과 병행해야 합니다.
    5. 투자 결정은 본인의 책임이며, 이 분석은 참고 자료일 뿐입니다.
    """)

🚀 MAG 7+2: Dark Pool & Short Interest 심층 분석 (표준 지표 적용)

데이터 수집 중... AAPL.. MSFT.. GOOGL.. AMZN.. NVDA.. META.. TSLA.. COIN.. IBIT.. 
완료!

📚 핵심 지표 체계 및 상세 해석 가이드

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 표준 공매도 지표 (Yahoo Finance 제공) - 가장 중요!     
       Short Float = (Number of Shorted Shares / Shares Float) × 100%
       Short Ratio = Number of Shorted Shares / Average Daily Trading Volume                                    │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 1️⃣ Short Ratio (Days to Cover) ⭐⭐⭐ 최우선 핵심 지표!                                              │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: 공매도 잔고 / 평균 일일 거래량                                                      │
   


📊 Chart 2: Short % of Float - 유통주식 대비 공매도 비율

    💡 Short % of Float 해석:

    🔍 의미:
    • 시장에서 거래 가능한 주식 중 몇 %가 공매도되었는가?
    • 시장 참여자들의 약세 심리 강도 측정

    📊 해석 기준:
    • <2%: 매우 낮음 (시장의 강한 신뢰)
    • 2-5%: 정상 범위 (건강한 시장)
    • 5-10%: 주의 (공매도 세력 관심 증가)
    • >10%: 높음 (강한 약세 베팅, Squeeze 잠재력도 높음)

    🎯 Days to Cover와 함께 보기:
    • Float >10% + DTC >5일 = 💥 극도로 위험(또는 기회!)
    • Float <5% + DTC <3일 = ✅ 안정적 종목
    



📊 Chart 3: 공매도 종합 비교 - Dark Pool vs 전체 시장

    💡 4가지 지표 종합 비교:

    1️⃣ DP Internal Short (파란색)
       • 장외 거래 중 공매도 비율
       • 기관의 은밀한 심리

    2️⃣ DP Market Impact (회색)
       • 전체 시장 대비 장외 공매도 영향
       • 절대적 규모

    3️⃣ FINRA/YF Ratio (보라색) ⭐
       • 오늘 발생한 공매도 / 전체 잔고
       • 공매도 "신선도" 측정
       • >50% = 신규 공매도 활발
       • <10% = 청산 진행

    📈 종합 해석:
    • DP Internal 높음 + FINRA/YF 높음 = 신규 공매도 공격
    • DP Internal 높음 + FINRA/YF 낮음 = 청산 시작 (기회!)
    



📊 Chart 4: Days to Cover vs Short % Float - 위험도 매트릭스

    💡 2D 위험도 분석:

    X축: Short % of Float (공매도 비율)
    Y축: Days to Cover (청산 소요 일수)

    🔥 우상단 (빨강): 극도 위험!
       • Float 높음 + DTC 높음
       • Short Squeeze 폭발 가능 구간
       • 가장 변동성 높은 영역

    💚 좌하단 (초록): 안정적
       • Float 낮음 + DTC 낮음
       • 공매도 압력 거의 없음
       • 펀더멘털 투자 적합

    버블 크기 = DP 내부 공매도 비율
    버블 색상 = Days to Cover (빨간색일수록 위험)
    



📊 Chart 5 & 6: 전체 종목 시계열 분석 - 60일 트렌드

    💡 시계열 차트 해석:

    1️⃣ 상단: DP 비중 Trend
       • 기관 개입 강도 변화
       • 상승 = 기관 활동 증가

    2️⃣ 중단: DP 내부 공매도 Trend + 10일 평균
       • 장외 공매도 심리 변화
       • 10일 평균 대비 ±5%p = 중요한 변곡점
       • 급락(-5%p) = 🟢 청산 신호
       • 급등(+5%p) = 🔴 공매도 공격

    📈 패턴 인식:
    • 중단 급락 = 공매도 청산 시작 (매수 기회)
    • 중단 급등 = 공매도 공격 시작 (주의)
    • 상단↑ + 중단↓ = 기관 매집
    • 상단↑ + 중단↑ = 기관 분산
    

🔍 AAPL (Apple) - DTC: 2.34일, Float: 0.78%


...


🔍 META (Meta) - DTC: 1.41일, Float: 1.39%



🔍 TSLA (Tesla) - DTC: 0.84일, Float: 2.45%



🔍 COIN (Coinbase) - DTC: 1.65일, Float: 6.92%



🔍 IBIT (Bitcoin ETF) - DTC: 0.00일, Float: 0.00%



✨ 최종 분석 요약 - 핵심 인사이트

✅ 건강한 종목 (DTC <3일 & Float <5%):
   • AAPL: DTC 2.34일, Float 0.78%
   • MSFT: DTC 2.14일, Float 0.65%
   • GOOGL: DTC 1.92일, Float 1.07%
   • META: DTC 1.41일, Float 1.39%
   • AMZN: DTC 1.37일, Float 0.73%
   • NVDA: DTC 1.27일, Float 1.02%
   • TSLA: DTC 0.84일, Float 2.45%
   • IBIT: DTC 0.00일, Float 0.00%

💚 기관 매집 가능성 (DP >50% & DP Short <45%):
   • NVDA: DP 50.6%, DP Short 39.7%

📊 데이터 출처:
   • Yahoo Finance: Days to Cover, Short % of Float (표준 지표)
   • FINRA: Dark Pool 거래 데이터 (장외 거래 분석)
🕐 분석 완료: 2025-11-30 11:34:56

💡 투자 시 주의사항:

    1. Days to Cover가 높다고 무조건 오르는 것은 아닙니다.
    2. 공매도 세력이 맞을 수도 있으며, 주가는 계속 하락할 수 있습니다.
    3. 하지만 일단 반등이 시작되면, DTC가 높을수록 Short Squeeze로 폭등할 가능성이 큽니다.
    4. 이 분석은 기술적 지표일 뿐, 펀더멘털 분석과 병행해야 합니다.
    5. 투자 결정은 본인의 책임이며, 이 분석은 참고 자료일 뿐입니다.
    


In [10]:
# 필요한 라이브러리 설치 및 임포트
# !pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== 설정 및 종목 리스트 ====================
MAG7_STOCKS = {
    'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Alphabet',
    'AMZN': 'Amazon', 'NVDA': 'NVIDIA', 'META': 'Meta',
    'TSLA': 'Tesla', 'COIN': 'Coinbase', 'IBIT': 'Bitcoin ETF'
}

# ==================== 데이터 수집 함수 ====================

def get_market_volume(ticker, days_back=65):
    """Yahoo Finance에서 전체 시장 거래량 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back + 10)
        df = stock.history(start=start_date, end=end_date)
        return df['Volume']
    except:
        return None

def get_yf_short_info(ticker):
    """Yahoo Finance에서 공매도 정보 가져오기 (표준 지표)"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        return {
            'shares_short': info.get('sharesShort', 0),  # 공매도 잔고
            'short_percent_float': info.get('shortPercentOfFloat', 0) * 100,  # Short % of Float
            'short_ratio_days': info.get('shortRatio', 0),  # Days to Cover ⭐
            'shares_outstanding': info.get('sharesOutstanding', 0)
        }
    except:
        return {
            'shares_short': 0,
            'short_percent_float': 0,
            'short_ratio_days': 0,
            'shares_outstanding': 0
        }

def get_finra_data_full(ticker, days_back=60):
    """
    FINRA 데이터 수집 및 핵심 지표 계산 (60일 history 포함)

    지표:
    1. dp_ratio: Dark Pool 비중 (FINRA 전체 / 시장 전체)
    2. dp_short_ratio: DP 내부 공매도 비율 (FINRA 공매도 / FINRA 전체)
    3. dp_short_market_impact: DP 공매도의 시장 영향 (FINRA 공매도 / 시장 전체)
    """
    try:
        today = datetime.now()
        data_list = []

        # Yahoo Finance 공매도 정보
        yf_short_info = get_yf_short_info(ticker)
        yf_shares_short = yf_short_info['shares_short']

        # Yahoo 전체 거래량
        market_volumes = get_market_volume(ticker, days_back)
        if market_volumes is None or market_volumes.empty:
            return None

        # FINRA 데이터 루프 (최근 N일)
        for days in range(days_back + 5):
            check_date = today - timedelta(days=days)
            if check_date.weekday() >= 5:
                continue

            date_str = check_date.strftime('%Y%m%d')
            date_key = check_date.strftime('%Y-%m-%d')

            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=3)
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text), sep='|')
                    if 'Symbol' in df.columns:
                        df.rename(columns={'Symbol': 'symbol'}, inplace=True)
                    if 'ShortVolume' in df.columns:
                        df.rename(columns={'ShortVolume': 'shortVolume'}, inplace=True)
                    if 'TotalVolume' in df.columns:
                        df.rename(columns={'TotalVolume': 'totalVolume'}, inplace=True)

                    row = df[df['symbol'] == ticker.upper()]

                    if not row.empty:
                        finra_total = int(row.iloc[0]['totalVolume'])
                        finra_short = int(row.iloc[0]['shortVolume'])

                        # Yahoo Volume 매칭
                        market_vol = 0
                        if date_key in market_volumes.index:
                            market_vol = market_volumes.loc[date_key]
                        else:
                            for idx in market_volumes.index:
                                if idx.strftime('%Y-%m-%d') == date_key:
                                    market_vol = market_volumes[idx]
                                    break

                        if market_vol > 0:
                            # 지표 계산
                            dp_ratio = (finra_total / market_vol) * 100
                            dp_short_ratio = (finra_short / finra_total) * 100
                            dp_short_market_impact = (finra_short / market_vol) * 100

                            if dp_ratio > 100:
                                dp_ratio = 100

                            data_list.append({
                                'date': date_key,
                                'dp_ratio': round(dp_ratio, 2),
                                'dp_short_ratio': round(dp_short_ratio, 2),
                                'dp_short_market_impact': round(dp_short_market_impact, 2),
                                'market_vol': market_vol,
                                'yf_shares_short': yf_shares_short
                            })

                if len(data_list) >= days_back:
                    break
            except:
                continue

        if not data_list:
            return None

        df_hist = pd.DataFrame(data_list).sort_values('date')
        df_hist['dp_short_ratio_10d_avg'] = df_hist['dp_short_ratio'].rolling(window=10, min_periods=1).mean()
        latest = df_hist.iloc[-1]
        recent_10d_avg = df_hist.iloc[-10:]['dp_short_ratio'].mean()
        dp_short_change = latest['dp_short_ratio'] - recent_10d_avg

        return {
            'ticker': ticker,
            'name': MAG7_STOCKS[ticker],
            'latest_date': latest['date'],
            'dp_ratio': latest['dp_ratio'],
            'dp_short_ratio': latest['dp_short_ratio'],
            'dp_short_market_impact': latest['dp_short_market_impact'],
            'dp_short_10d_avg': latest['dp_short_ratio_10d_avg'],
            'dp_short_change_pct': dp_short_change,
            'yf_shares_short': latest['yf_shares_short'],
            'yf_short_percent_float': yf_short_info['short_percent_float'],
            'yf_short_ratio_days': yf_short_info['short_ratio_days'],  # ⭐ Days to Cover
            'history': df_hist
        }

    except Exception as e:
        return None

# ==================== 메인 분석 실행 ====================

print("="*100)
print("🚀 MAG 7+2: Dark Pool & Short Interest 심층 분석 (표준 지표 적용)")
print("="*100)

analysis_results = []
print("\n데이터 수집 중...", end=" ")

for ticker in MAG7_STOCKS.keys():
    print(f"{ticker}..", end=" ")
    res = get_finra_data_full(ticker, days_back=60)
    if res:
        analysis_results.append(res)

print("\n완료!")

if not analysis_results:
    print("❌ 데이터를 가져올 수 없습니다.")
else:
    # Calculate finra_yf_short_ratio for each item in analysis_results before creating df_main
    for item in analysis_results:
        yf_shares_short = item['yf_shares_short']
        if yf_shares_short > 0 and not item['history'].empty:
            # Reconstruct daily FINRA short volume from dp_short_market_impact and latest market_vol
            latest_market_vol = item['history'].iloc[-1]['market_vol']
            daily_finra_short_vol = (item['dp_short_market_impact'] * latest_market_vol) / 100

            finra_yf_short_ratio = (daily_finra_short_vol / yf_shares_short) * 100
            item['finra_yf_short_ratio'] = finra_yf_short_ratio
        else:
            item['finra_yf_short_ratio'] = 0.0 # Assign 0 if yf_shares_short is 0 or history is empty

    df_main = pd.DataFrame([{k:v for k,v in r.items() if k != 'history'} for r in analysis_results])

    # 시그널 컬럼 생성
    def create_signal(row):
        # 1. Short Squeeze 임박 (Days to Cover 높음 + DP Short 급락)
        if row['yf_short_ratio_days'] > 5 and row['dp_short_change_pct'] < -5:
            return '🔥 Short Squeeze 임박!'

        # 2. 공매도 청산 신호
        if row['dp_short_change_pct'] < -5:
            return '🟢 급락 (청산 신호)'

        # 3. Days to Cover 매우 높음
        if row['yf_short_ratio_days'] > 7:
            return '🔴🔴 극심한 공매도 (7일+)'

        # 4. 기관 강한 약세
        if row['dp_ratio'] > 50 and row['dp_short_ratio'] > 55:
            return '🔴 기관 강한 약세'

        # 5. 기관 매집
        if row['dp_ratio'] > 50 and row['dp_short_ratio'] < 45:
            return '💚 기관 매집 가능성'

        # 6. Days to Cover 정상
        if row['yf_short_ratio_days'] < 3:
            return '✅ 건강 (DTC <3일)'

        return '⚪ 관망/정상'

    df_main['Signal'] = df_main.apply(create_signal, axis=1)
    df_main = df_main.sort_values('yf_short_ratio_days', ascending=False)

    # ==================== 핵심 지표 해석 가이드 ====================

    print("\n" + "="*120)
    print("📚 핵심 지표 체계 및 상세 해석 가이드")
    print("="*120)
    print("""
    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 표준 공매도 지표 (Yahoo Finance 제공) - 가장 중요!                                          │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 1️⃣ Short Ratio (Days to Cover) ⭐⭐⭐ 최우선 핵심 지표!                                              │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: 공매도 잔고 / 평균 일일 거래량                                                      │
    │                                                                                                │
    │ 💡 의미:                                                                                       │
    │    • 현재 쌓여있는 모든 공매도 포지션을 청산하는데 며칠 걸리는가?                             │
    │    • 공매도 세력이 "탈출"하는데 필요한 시간                                                   │
    │    • Short Squeeze 위험도의 직접적 측정치                                                     │
    │                                                                                                │
    │ 📊 해석 기준:                                                                                  │
    │    • <3일 (초록) : 정상 - 공매도 청산 용이, 위험 낮음                                         │
    │    • 3-5일 (주황): 주의 - 공매도 압력 존재, 변동성 가능                                       │
    │    • 5-7일 (빨강): 높음 - Short Squeeze 가능성, 급등 잠재력                                   │
    │    • >7일 (적색) : 매우 높음 - Short Squeeze 고위험, 폭발적 상승 가능                         │
    │                                                                                                │
    │ 🎯 실전 활용:                                                                                  │
    │    • Days to Cover >5일 + 호재 발생 = 🔥 Short Squeeze 폭발!                                 │
    │    • Days to Cover >7일 + DP Short 급락(-5%p) = 🚀 청산 시작, 강력 매수 신호               │
    │    • Days to Cover <3일 = 공매도 압력 적음, 안정적 거래 가능                                  │
    │                                                                                                │
    │ ⚠️ 주의사항:                                                                                   │
    │    • 높다고 무조건 좋은 것은 아님! 공매도 세력이 맞을 수도 있음                               │
    │    • 하지만 일단 상승 시작하면 → 청산 압력으로 폭등 가능                                      │
    │    • GameStop(GME) 사태가 대표적 예시 (Days to Cover 매우 높았음)                            │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 2️⃣ Short % of Float (Short Float) ⭐⭐                                                         │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (공매도 잔고 / 유통주식수) × 100                                                    │
    │                                                                                                │
    │ 💡 의미:                                                                                       │
    │    • 시장에서 거래 가능한 주식(Float) 중 몇 %가 공매도되었는가?                               │
    │    • 시장 참여자들의 약세 심리 강도                                                           │
    │    • 유통주식 대비 공매도 비중                                                                │
    │                                                                                                │
    │ 📊 해석 기준:                                                                                  │
    │    • <2% (초록)  : 매우 낮음 - 시장의 강한 신뢰                                               │
    │    • 2-5% (연두) : 정상 - 일반적인 수준                                                       │
    │    • 5-10% (주황): 주의 - 공매도 세력의 관심 증가                                             │
    │    • >10% (빨강) : 높음 - 강한 약세 베팅 + Short Squeeze 잠재력                              │
    │    • >20% (적색) : 매우 높음 - 극단적 약세 베팅                                               │
    │                                                                                                │
    │ 🎯 실전 활용:                                                                                  │
    │    • Float >10% + Days to Cover >5일 = 💣 최고 위험도 (or 기회)                             │
    │    • Float <5% = 건강한 종목, 공매도 압력 낮음                                                │
    │                                                                                                │
    │ ⚠️ Days to Cover와의 차이:                                                                    │
    │    • Short Float: 유통주식 대비 비율 (정적 지표, 장기 관점)                                  │
    │    • Days to Cover: 거래량 대비 청산 일수 (동적 지표, 위험도 측정)                           │
    │    • 두 지표 모두 높으면 → 극도로 위험한(또는 기회가 되는) 상황!                             │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 FINRA 장외 거래 지표 (Dark Pool Analysis)                                                   │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 3️⃣ DP Ratio (Dark Pool 비중) ⭐                                                                │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 전체 거래량 / 전체 시장 거래량) × 100                                        │
    │ 💡 의미: 전체 시장에서 장외(비공개) 거래가 차지하는 비중                                       │
    │ 📊 해석: >50% = 기관 과열, 40-50% = 기관 관심, <40% = 정상                                   │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 4️⃣ DP Internal Short Ratio (DP 내부 공매도) ⭐                                                │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 공매도량 / FINRA 전체 거래량) × 100                                          │
    │ 💡 의미: 장외 거래 중 공매도가 차지하는 비율 (기관의 숨겨진 심리)                              │
    │ 📊 해석: >55% = 강한 약세, 45-55% = 중립, <45% = 강세                                         │
    │ 🎯 활용: DP Ratio와 함께 봐야 정확! DP↑ + DP Short↓ = 매집, DP↑ + DP Short↑ = 분산        │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 5️⃣ DP Short Market Impact (DP 공매도 시장 영향)                                               │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 공매도량 / 전체 시장 거래량) × 100                                           │
    │ 💡 의미: 전체 시장 거래량 대비 장외 공매도의 실제 영향력                                       │
    │ 📊 해석: 장외 공매도가 전체 시장에 미치는 절대적 규모                                          │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 6️⃣ FINRA/YF Short Ratio (공매도 신선도) - 우리의 독자 지표                                    │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: (FINRA 일일 공매도량 / YF 전체 공매도 잔고) × 100                                   │
    │ 💡 의미: 전체 공매도 중 오늘 장외에서 발생한 비율 (신규 vs 기존)                              │
    │ 📊 해석: >50% = 활발한 신규 공매도, 10-50% = 정상, <10% = 청산 진행 중                       │
    │ 🎯 활용: Days to Cover >5일 + FINRA/YF <10% = 🟢 청산 시작! (매수 기회)                     │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘
    """)
    print("="*120)

    # ==================== 종합 시나리오 분석 ====================

    print("\n" + "="*120)
    print("🎯 종합 시나리오 분석 - 지표 조합으로 상황 파악하기")
    print("="*120)
    print("""
    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 🔥 시나리오 1: Short Squeeze 임박! (최고 기회 or 최고 위험)                                    │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover > 5일 (공매도 청산 어려움)                                                    │
    │  • Short % Float > 10% (높은 공매도 비율)                                                      │
    │  • DP Short 급락 (10일 대비 -5%p 이상)                                                         │
    │  • FINRA/YF < 10% (신규 공매도 없음, 청산 시작)                                                │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 세력이 쌓아놓은 포지션은 많은데(Float >10%, DTC >5일),                                 │
    │  청산하기 시작했고(DP Short 급락, FINRA/YF 낮음),                                              │
    │  청산에 시간도 오래 걸림(DTC >5일)                                                             │
    │  → 💥 연쇄 청산으로 폭등 가능성!                                                              │
    │                                                                                                │
    │ 🎯 전략: 적극적 매수 진입 (단, 리스크 높음!)                                                   │
    │                                                                                                │
    │ 📌 역사적 사례: GameStop (GME), AMC 등                                                         │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 🔴 시나리오 2: 공매도 공격 진행 중                                                             │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover 증가 추세                                                                     │
    │  • DP Short > 55% (장외에서 강한 공매도)                                                       │
    │  • DP Short 급등 (10일 대비 +5%p 이상)                                                         │
    │  • FINRA/YF > 50% (활발한 신규 공매도)                                                         │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 세력이 적극적으로 포지션을 늘리고 있음                                                 │
    │  → 하락 압력 증가 예상                                                                        │
    │                                                                                                │
    │ 🎯 전략: 관망 또는 단기 숏 포지션 (역추세 매수는 위험)                                         │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 💚 시나리오 3: 기관 매집 (장외 매수)                                                           │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • DP Ratio > 50% (기관 개입 강함)                                                             │
    │  • DP Short < 45% (장외에서 매수 우위)                                                         │
    │  • Days to Cover < 3일 (공매도 압력 낮음)                                                      │
    │  • Short % Float < 5% (건강한 수준)                                                            │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  기관들이 장외에서 조용히 매수 중, 공매도 압력도 낮음                                          │
    │  → 안정적 상승 가능성                                                                         │
    │                                                                                                │
    │ 🎯 전략: 안정적 매수 또는 홀딩                                                                 │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ ✅ 시나리오 4: 건강한 종목 (이상적)                                                            │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover < 3일                                                                         │
    │  • Short % Float < 5%                                                                          │
    │  • DP Short 40-50% (중립)                                                                      │
    │  • FINRA/YF 10-50% (정상 회전)                                                                 │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 압력 낮고, 기관 활동도 정상 범위                                                       │
    │  → 안정적 거래 가능, 변동성 낮음                                                              │
    │                                                                                                │
    │ 🎯 전략: 펀더멘털 분석 기반 투자                                                               │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ ⚠️ 시나리오 5: 주의 필요 (공매도 많지만 청산은 안됨)                                           │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 조건:                                                                                          │
    │  • Days to Cover > 7일 (매우 높음)                                                             │
    │  • Short % Float > 15%                                                                         │
    │  • BUT, DP Short 변화 없음 (횡보)                                                              │
    │  • FINRA/YF 10-50% (정상 회전)                                                                 │
    │                                                                                                │
    │ 📊 해석:                                                                                       │
    │  공매도 잔고는 많지만 청산도, 추가 공격도 없는 교착 상태                                       │
    │  → 촉매(호재/악재) 기다리는 중, 어느 방향으로든 큰 움직임 가능                                 │
    │                                                                                                │
    │ 🎯 전략: 대기, 촉매 확인 후 대응                                                               │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘
    """)
    print("="*120)

    # ==================== 통합 상세 비교 테이블 ====================

    print("\n" + "="*150)
    print("📋 통합 상세 비교 테이블: 표준 지표 중심 분석")
    print("="*150)
    print("""
    💡 테이블 읽는 법:
    • Days_to_Cover >5일 = Short Squeeze 위험 구간
    • Short_%_Float >10% = 높은 공매도 비율
    • DP내부공매도 >55% = 장외에서 강한 약세
    • 1일vs10일 <-5%p = 청산 시작 신호
    • FINRA/YF >50% = 활발한 신규 공매도, <10% = 청산 진행
    """)

    df_display = df_main.copy()
    df_display = df_display.rename(columns={
        'ticker': '티커',
        'name': '종목명',
        'yf_short_ratio_days': 'Days_to_Cover',
        'yf_short_percent_float': 'Short_%_Float',
        'dp_ratio': 'DP비중_%',
        'dp_short_ratio': 'DP내부공매도_%',
        'dp_short_10d_avg': 'DP_10일평균',
        'dp_short_change_pct': '1일vs10일',
        'dp_short_market_impact': 'DP→시장_%',
        'finra_yf_short_ratio': 'FINRA/YF_%',
        'Signal': '신호'
    })

    table_cols = ['티커', '종목명', 'Days_to_Cover', 'Short_%_Float',
                  'DP비중_%', 'DP내부공매도_%', 'DP_10일평균', '1일vs10일',
                  'DP→시장_%', 'FINRA/YF_%', '신호']

    # 포맷팅
    df_display['Days_to_Cover'] = df_display['Days_to_Cover'].apply(lambda x: f"{x:.2f}일")
    df_display['Short_%_Float'] = df_display['Short_%_Float'].apply(lambda x: f"{x:.2f}%")
    df_display['DP비중_%'] = df_display['DP비중_%'].apply(lambda x: f"{x:.2f}%")
    df_display['DP내부공매도_%'] = df_display['DP내부공매도_%'].apply(lambda x: f"{x:.2f}%")
    df_display['DP_10일평균'] = df_display['DP_10일평균'].apply(lambda x: f"{x:.2f}%")
    df_display['1일vs10일'] = df_display['1일vs10일'].apply(lambda x: f"{x:+.2f}%p")
    df_display['DP→시장_%'] = df_display['DP→시장_%'].apply(lambda x: f"{x:.2f}%")
    df_display['FINRA/YF_%'] = df_display['FINRA/YF_%'].apply(lambda x: f"{x:.1f}%")

    print(df_display[table_cols].to_string(index=False))
    print("="*150)

    # ==================== 요약 통계 ====================

    print("\n" + "="*100)
    print("📊 전체 시장 개요")
    print("="*100)
    print(f"평균 Days to Cover:    {df_main['yf_short_ratio_days'].mean():.2f}일")
    print(f"평균 Short % Float:    {df_main['yf_short_percent_float'].mean():.2f}%")
    print(f"평균 DP 비중:          {df_main['dp_ratio'].mean():.2f}%")
    print(f"평균 DP 내부 공매도:   {df_main['dp_short_ratio'].mean():.2f}%")
    print()
    print(f"Short Squeeze 위험 (DTC >5일):        {len(df_main[df_main['yf_short_ratio_days'] > 5])}개")
    print(f"높은 공매도 비율 (Float >10%):        {len(df_main[df_main['yf_short_percent_float'] > 10])}개")
    print(f"청산 신호 (1일vs10일 <-5%p):           {len(df_main[df_main['dp_short_change_pct'] < -5])}개")
    print(f"활발한 신규 공매도 (FINRA/YF >50%):    {len(df_main[df_main['finra_yf_short_ratio'] > 50])}개")
    print("="*100)

    # ==================== 차트 1: Days to Cover (가장 중요!) ====================

    print("\n" + "="*120)
    print("📊 Chart 1: Days to Cover - Short Squeeze 위험도 측정 ⭐⭐⭐ 최우선 지표!")
    print("="*120)
    print("""
    💡 Days to Cover (청산 소요 일수) 상세 해석:

    🔍 이 지표가 가장 중요한 이유:
    • Short Squeeze 가능성을 직접적으로 측정
    • 공매도 세력의 "탈출 난이도" 표시
    • 변동성 및 급등 가능성 예측

    📊 해석 기준 (매우 중요!):
    • <3일 (초록): 정상
      → 공매도 세력이 빠르게 청산 가능
      → 변동성 낮음, 안정적 거래
      → 공매도 압력 미미

    • 3-5일 (노랑): 주의
      → 공매도 청산에 시간 소요
      → 호재 발생 시 변동성 증가 가능
      → 모니터링 필요

    • 5-7일 (주황): 위험
      → Short Squeeze 가능성 있음
      → 공매도 세력 탈출 어려움
      → 급등 잠재력 존재

    • >7일 (빨강): 극도 위험!
      → Short Squeeze 고위험 구간
      → 한 번 상승 시작하면 연쇄 청산으로 폭등 가능
      → GameStop(GME) 사태가 대표적 예시 (Days to Cover 매우 높았음)

    🎯 실전 활용 전략:
    1. DTC >7일 + 호재 발생 = 🚀 폭발적 상승 가능
    2. DTC >5일 + DP Short 급락 = 🔥 청산 시작, 매수 기회
    3. DTC <3일 = 안정적 종목, 펀더멘털 위주 투자

    ⚠️ 함정 주의:
    • DTC가 높다고 무조건 오르는 것은 아님!
    • 공매도 세력이 맞을 수도 있음 (주가 계속 하락 가능)
    • 하지만 일단 반등 시작하면 → 청산 압력으로 급등!
    """)

    fig1 = go.Figure()

    # Days to Cover 기준 색상
    colors_dtc = []
    for x in df_main['yf_short_ratio_days']:
        if x < 3:
            colors_dtc.append('green')
        elif x < 5:
            colors_dtc.append('yellow')
        elif x < 7:
            colors_dtc.append('orange')
        else:
            colors_dtc.append('red')

    fig1.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['yf_short_ratio_days'],
        text=[f"{x:.2f}일" for x in df_main['yf_short_ratio_days']],
        textposition='auto',
        marker_color=colors_dtc,
        hovertemplate='<b>%{x}</b><br>Days to Cover: %{y:.2f}일<br>청산 소요 기간<extra></extra>'
    ))

    fig1.add_hline(y=3, line_dash="dash", line_color="green", annotation_text="정상 (3일)")
    fig1.add_hline(y=5, line_dash="dash", line_color="orange", annotation_text="주의 (5일)")
    fig1.add_hline(y=7, line_dash="dash", line_color="red", annotation_text="위험 (7일)")

    fig1.update_layout(
        title='Days to Cover (Short Ratio): 공매도 청산 소요 일수 - Short Squeeze 핵심 지표',
        height=550,
        template='plotly_white',
        xaxis_title='종목',
        yaxis_title='Days to Cover (일)'
    )
    fig1.show()

    # ==================== 차트 2: Short % of Float ====================

    print("\n" + "="*120)
    print("📊 Chart 2: Short % of Float - 유통주식 대비 공매도 비율")
    print("="*120)
    print("""
    💡 Short % of Float 해석:

    🔍 의미:
    • 시장에서 거래 가능한 주식 중 몇 %가 공매도되었는가?
    • 시장 참여자들의 약세 심리 강도 측정

    📊 해석 기준:
    • <2%: 매우 낮음 (시장의 강한 신뢰)
    • 2-5%: 정상 범위 (건강한 시장)
    • 5-10%: 주의 (공매도 세력 관심 증가)
    • >10%: 높음 (강한 약세 베팅, Squeeze 잠재력도 높음)

    🎯 Days to Cover와 함께 보기:
    • Float >10% + DTC >5일 = 💥 극도로 위험(또는 기회!)
    • Float <5% + DTC <3일 = ✅ 안정적 종목
    """)

    fig2 = go.Figure()

    colors_float = ['green' if x < 2 else 'yellowgreen' if x < 5 else 'orange' if x < 10 else 'red'
                   for x in df_main['yf_short_percent_float']]

    fig2.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['yf_short_percent_float'],
        text=[f"{x:.2f}%" for x in df_main['yf_short_percent_float']],
        textposition='auto',
        marker_color=colors_float,
        hovertemplate='<b>%{x}</b><br>Short % Float: %{y:.2f}%<extra></extra>'
    ))

    fig2.add_hline(y=2, line_dash="dash", line_color="green", annotation_text="매우 낮음 (2%)")
    fig2.add_hline(y=5, line_dash="dash", line_color="yellowgreen", annotation_text="정상 (5%)")
    fig2.add_hline(y=10, line_dash="dash", line_color="red", annotation_text="높음 (10%)")

    fig2.update_layout(
        title='Short % of Float: 유통주식 대비 공매도 비율',
        xaxis_title='종목',
        yaxis_title='Short % of Float (%)',
        height=550,
        template='plotly_white'
    )
    fig2.show()

    # ==================== 차트 2-1: DP Ratio (Dark Pool 비중) ====================

    print("\n" + "="*120)
    print("📊 Chart 2-1: DP Ratio - Dark Pool 거래 비중")
    print("="*120)
    print("""
    💡 DP Ratio (Dark Pool 비중) 해석:

    🔍 의미:
    • 전체 시장 거래량 중 장외(Dark Pool)에서 거래된 비중
    • 기관 투자자들의 시장 개입 강도 측정
    • 높을수록 기관들이 '은밀하게' 거래 중

    📊 해석 기준:
    • <40%: 정상 범위 (일반적인 시장 거래)
    • 40-50%: 기관 관심 증가 (모니터링 필요)
    • >50%: 기관 과열 (강한 기관 개입)
    • >60%: 극도 과열 (비정상적 수준)

    🎯 실전 활용:
    • DP Ratio >50% + DP Short >55% = 기관 강한 약세
    • DP Ratio >50% + DP Short <45% = 기관 매집 가능성
    • DP Ratio <40% = 정상 시장, 기관 개입 낮음
    """)

    fig2_1 = go.Figure()

    # DP Ratio 기준 색상
    colors_dp = []
    for x in df_main['dp_ratio']:
        if x < 40:
            colors_dp.append('green')
        elif x < 50:
            colors_dp.append('yellowgreen')
        elif x < 60:
            colors_dp.append('orange')
        else:
            colors_dp.append('red')

    fig2_1.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['dp_ratio'],
        text=[f"{x:.1f}%" for x in df_main['dp_ratio']],
        textposition='auto',
        marker_color=colors_dp,
        hovertemplate='<b>%{x}</b><br>DP Ratio: %{y:.2f}%<br>장외 거래 비중<extra></extra>'
    ))

    fig2_1.add_hline(y=40, line_dash="dash", line_color="green", annotation_text="정상 (40%)")
    fig2_1.add_hline(y=50, line_dash="dash", line_color="orange", annotation_text="과열 (50%)")
    fig2_1.add_hline(y=60, line_dash="dash", line_color="red", annotation_text="극도과열 (60%)")

    fig2_1.update_layout(
        title='DP Ratio (Dark Pool 비중): 전체 시장 대비 장외 거래 비중',
        height=550,
        template='plotly_white',
        xaxis_title='종목',
        yaxis_title='DP Ratio (%)'
    )
    fig2_1.show()

    # ==================== 차트 3: 공매도 종합 비교 ====================

    print("\n" + "="*120)
    print("📊 Chart 3: 공매도 종합 비교 - Dark Pool vs 전체 시장")
    print("="*120)
    print("""
    💡 4가지 지표 종합 비교:

    1️⃣ DP Internal Short (파란색)
       • 장외 거래 중 공매도 비율
       • 기관의 은밀한 심리

    2️⃣ DP Market Impact (회색)
       • 전체 시장 대비 장외 공매도 영향
       • 절대적 규모

    3️⃣ FINRA/YF Ratio (보라색) ⭐
       • 오늘 발생한 공매도 / 전체 잔고
       • 공매도 "신선도" 측정
       • >50% = 신규 공매도 활발
       • <10% = 청산 진행

    📈 종합 해석:
    • DP Internal 높음 + FINRA/YF 높음 = 신규 공매도 공격
    • DP Internal 높음 + FINRA/YF 낮음 = 청산 시작 (기회!)
    """)

    fig3 = go.Figure()

    # DP 내부 공매도
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['dp_short_ratio'],
        name='DP Internal Short',
        marker_color='darkblue',
        text=df_main['dp_short_ratio'].round(1),
        textposition='auto'
    ))

    # DP→시장 공매도
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['dp_short_market_impact'],
        name='DP Market Impact',
        marker_color='gray',
        text=df_main['dp_short_market_impact'].round(1),
        textposition='auto'
    ))

    # FINRA/YF 비율
    fig3.add_trace(go.Bar(
        x=df_main['ticker'],
        y=df_main['finra_yf_short_ratio'],
        name='FINRA/YF Ratio (신선도)',
        marker_color='purple',
        text=df_main['finra_yf_short_ratio'].round(1),
        textposition='auto'
    ))

    fig3.add_hline(y=50, line_dash="dash", line_color="orange", annotation_text="50% 기준")

    fig3.update_layout(
        title='공매도 종합 비교: DP Internal vs Market Impact vs 신선도',
        barmode='group',
        height=550,
        template='plotly_white',
        xaxis_title='종목',
        yaxis_title='비율 (%)',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig3.show()

    # ==================== 차트 4: Days to Cover vs Short % Float ====================

    print("\n" + "="*120)
    print("📊 Chart 4: Days to Cover vs Short % Float - 위험도 매트릭스")
    print("="*120)
    print("""
    💡 2D 위험도 분석:

    X축: Short % of Float (공매도 비율)
    Y축: Days to Cover (청산 소요 일수)

    🔥 우상단 (빨강): 극도 위험!
       • Float 높음 + DTC 높음
       • Short Squeeze 폭발 가능 구간
       • 가장 변동성 높은 영역

    💚 좌하단 (초록): 안정적
       • Float 낮음 + DTC 낮음
       • 공매도 압력 거의 없음
       • 펀더멘털 투자 적합

    버블 크기 = DP 내부 공매도 비율
    버블 색상 = Days to Cover (빨간색일수록 위험)
    """)

    fig4 = go.Figure()

    fig4.add_trace(go.Scatter(
        x=df_main['yf_short_percent_float'],
        y=df_main['yf_short_ratio_days'],
        mode='markers+text',
        text=df_main['ticker'],
        textposition='top center',
        marker=dict(
            size=df_main['dp_short_ratio'] * 1.2,
            color=df_main['yf_short_ratio_days'],
            colorscale='RdYlGn_r',
            showscale=True,
            colorbar=dict(title="Days to Cover"),
            line=dict(width=1, color='black')
        ),
        hovertemplate='<b>%{text}</b><br>Float: %{x:.2f}%<br>DTC: %{y:.2f}일<extra></extra>'
    ))

    fig4.add_vline(x=10, line_dash="dot", line_color="gray", line_width=2)
    fig4.add_hline(y=5, line_dash="dot", line_color="gray", line_width=2)

    fig4.add_annotation(x=15, y=8, text="<b>🔥 극도 위험<br>Squeeze Zone</b>",
                      showarrow=False, font=dict(color="darkred", size=12),
                      bgcolor="rgba(255,200,200,0.3)", bordercolor="red", borderwidth=2, borderpad=4)
    fig4.add_annotation(x=3, y=2, text="<b>💚 안정적<br>Safe Zone</b>",
                      showarrow=False, font=dict(color="darkgreen", size=12),
                      bgcolor="rgba(200,255,200,0.3)", bordercolor="green", borderwidth=2, borderpad=4)

    fig4.update_layout(
        title='Short Squeeze Risk Matrix: Float % vs Days to Cover',
        xaxis_title='Short % of Float (%)',
        yaxis_title='Days to Cover (일)',
        height=600,
        template='plotly_white'
    )
    fig4.show()

    # ==================== 차트 4-1: DP Ratio vs DP Short Ratio 4분면 분석 ====================

    print("\n" + "="*120)
    print("📊 Chart 4-1: DP Ratio vs DP Short Ratio - 기관 포지션 매트릭스")
    print("="*120)
    print("""
    💡 기관 투자자 포지션 4분면 분석:

    X축: DP Ratio (장외 거래 비중) - 기관 개입 강도
    Y축: DP Short Ratio (장외 내부 공매도 비율) - 기관 심리

    🔴 우상단 (빨강): 기관 강한 약세
       • DP Ratio >50% (강한 기관 개입)
       • DP Short >55% (장외에서 공매도 우위)
       • 해석: 기관들이 적극적으로 공매도 중
       • 전략: 하락 압력 주의, 역추세 매수 위험

    💚 우하단 (초록): 기관 매집 가능성
       • DP Ratio >50% (강한 기관 개입)
       • DP Short <45% (장외에서 매수 우위)
       • 해석: 기관들이 조용히 매수 중
       • 전략: 상승 잠재력, 안정적 매수 기회

    ⚪ 좌측 (회색): 정상 범위
       • DP Ratio <50% (기관 개입 낮음)
       • 해석: 일반 시장 거래 우세
       • 전략: 펀더멘털 중심 투자

    버블 크기 = Days to Cover (클수록 Short Squeeze 위험)
    버블 색상 = DP Short Ratio (빨간색일수록 약세)
    """)

    fig4_1 = go.Figure()

    # 버블 차트 생성
    fig4_1.add_trace(go.Scatter(
        x=df_main['dp_ratio'],
        y=df_main['dp_short_ratio'],
        mode='markers+text',
        text=df_main['ticker'],
        textposition='top center',
        marker=dict(
            size=df_main['yf_short_ratio_days'] * 8,  # Days to Cover 반영
            color=df_main['dp_short_ratio'],  # DP Short Ratio로 색상
            colorscale='RdYlGn_r',
            showscale=True,
            colorbar=dict(title="DP Short<br>Ratio (%)"),
            line=dict(width=1, color='black'),
            cmin=40,
            cmax=60
        ),
        hovertemplate='<b>%{text}</b><br>DP Ratio: %{x:.1f}%<br>DP Short: %{y:.1f}%<br>DTC: ' +
                      df_main['yf_short_ratio_days'].round(2).astype(str) + '일<extra></extra>'
    ))

    # 4분면 구분선
    fig4_1.add_vline(x=50, line_dash="dot", line_color="gray", line_width=2)
    fig4_1.add_hline(y=50, line_dash="dot", line_color="gray", line_width=2)

    # 추가 기준선
    fig4_1.add_vline(x=40, line_dash="dash", line_color="lightgray", line_width=1)
    fig4_1.add_hline(y=45, line_dash="dash", line_color="lightgray", line_width=1)
    fig4_1.add_hline(y=55, line_dash="dash", line_color="lightgray", line_width=1)

    # 4분면 라벨링
    fig4_1.add_annotation(
        x=60, y=60,
        text="<b>🔴 기관 강한 약세<br>Active Short</b>",
        showarrow=False,
        font=dict(color="darkred", size=12),
        bgcolor="rgba(255,200,200,0.3)",
        bordercolor="red",
        borderwidth=2,
        borderpad=4
    )

    fig4_1.add_annotation(
        x=60, y=40,
        text="<b>💚 기관 매집<br>Accumulation</b>",
        showarrow=False,
        font=dict(color="darkgreen", size=12),
        bgcolor="rgba(200,255,200,0.3)",
        bordercolor="green",
        borderwidth=2,
        borderpad=4
    )

    fig4_1.add_annotation(
        x=35, y=50,
        text="<b>⚪ 정상 범위<br>Normal Market</b>",
        showarrow=False,
        font=dict(color="gray", size=11),
        bgcolor="rgba(240,240,240,0.3)",
        bordercolor="gray",
        borderwidth=1,
        borderpad=4
    )

    fig4_1.update_layout(
        title='기관 포지션 매트릭스: DP Ratio vs DP Short Ratio<br><sub>버블 크기 = Days to Cover (Short Squeeze 위험도)</sub>',
        xaxis_title='DP Ratio (%) - 장외 거래 비중',
        yaxis_title='DP Short Ratio (%) - 장외 내부 공매도 비율',
        height=650,
        template='plotly_white',
        xaxis=dict(range=[30, 70]),
        yaxis=dict(range=[35, 65])
    )

    fig4_1.show()

    # ==================== 차트 5 & 6: 시계열 분석 ====================

    print("\n" + "="*120)
    print("📊 Chart 5 & 6: 전체 종목 시계열 분석 - 60일 트렌드")
    print("="*120)
    print("""
    💡 시계열 차트 해석:

    1️⃣ 상단: DP 비중 Trend
       • 기관 개입 강도 변화
       • 상승 = 기관 활동 증가

    2️⃣ 중단: DP 내부 공매도 Trend + 10일 평균
       • 장외 공매도 심리 변화
       • 10일 평균 대비 ±5%p = 중요한 변곡점
       • 급락(-5%p) = 🟢 청산 신호
       • 급등(+5%p) = 🔴 공매도 공격

    📈 패턴 인식:
    • 중단 급락 = 공매도 청산 시작 (매수 기회)
    • 중단 급등 = 공매도 공격 시작 (주의)
    • 상단↑ + 중단↓ = 기관 매집
    • 상단↑ + 중단↑ = 기관 분산
    """)

    for item in analysis_results:
        ticker = item['ticker']
        name = item['name']
        df_hist = item['history']

        print(f"\n🔍 {ticker} ({name}) - DTC: {item['yf_short_ratio_days']:.2f}일, Float: {item['yf_short_percent_float']:.2f}%")

        fig_ts = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            subplot_titles=(
                f"{ticker} - Dark Pool 비중 Trend",
                f"{ticker} - DP 내부 공매도 Trend + 10일 평균"
            ),
            vertical_spacing=0.12
        )

        # 차트 1: DP 비중
        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_ratio'],
            mode='lines+markers',
            name='DP 비중',
            line=dict(color='blue', width=2),
            showlegend=False,
            hovertemplate='날짜: %{x}<br>DP비중: %{y:.2f}%<extra></extra>'
        ), row=1, col=1)

        fig_ts.add_hline(y=50, line_dash="dot", line_color="red",
                        annotation_text="과열 (50%)", row=1, col=1)

        # 차트 2: DP 내부 공매도 + 10일 평균
        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_short_ratio'],
            mode='lines+markers',
            name='DP 내부 공매도',
            line=dict(color='orange', width=2),
            showlegend=False,
            hovertemplate='DP Short: %{y:.2f}%<extra></extra>'
        ), row=2, col=1)

        fig_ts.add_trace(go.Scatter(
            x=df_hist['date'],
            y=df_hist['dp_short_ratio_10d_avg'],
            mode='lines',
            name='10일 평균',
            line=dict(color='gray', dash='dot', width=1.5),
            showlegend=False,
            hovertemplate='10일평균: %{y:.2f}%<extra></extra>'
        ), row=2, col=1)

        fig_ts.add_hline(y=50, line_dash="dot", line_color="gray",
                        annotation_text="분기점", row=2, col=1)

        # 급락/급등 구간 하이라이트
        for i in range(1, len(df_hist)):
            prev = df_hist.iloc[i-1]
            curr = df_hist.iloc[i]
            change = curr['dp_short_ratio'] - prev['dp_short_ratio']

            if change < -5:  # 급락 (녹색)
                fig_ts.add_vrect(
                    x0=prev['date'], x1=curr['date'],
                    fillcolor="green", opacity=0.15,
                    layer="below", line_width=0,
                    row=2, col=1
                )
            elif change > 5:  # 급등 (빨간색)
                fig_ts.add_vrect(
                    x0=prev['date'], x1=curr['date'],
                    fillcolor="red", opacity=0.15,
                    layer="below", line_width=0,
                    row=2, col=1
                )

        fig_ts.update_layout(
            height=700,
            title_text=f"📊 {ticker} ({name}) - 60일 트렌드 | DTC: {item['yf_short_ratio_days']:.2f}일",
            template='plotly_white',
            hovermode='x unified'
        )

        fig_ts.update_xaxes(title_text="날짜", row=2, col=1)
        fig_ts.update_yaxes(title_text="DP 비중 (%)", row=1, col=1)
        fig_ts.update_yaxes(title_text="DP 내부 공매도 (%)", row=2, col=1)

        fig_ts.show()

    # ==================== 최종 요약 및 인사이트 ====================

    print("\n" + "="*120)
    print("✨ 최종 분석 요약 - 핵심 인사이트")
    print("="*120)

    # 1. Short Squeeze 고위험 종목
    squeeze_risk = df_main[(df_main['yf_short_ratio_days'] > 5) & (df_main['yf_short_percent_float'] > 10)]
    if not squeeze_risk.empty:
        print(f"\n🔥 Short Squeeze 고위험 종목 (DTC >5일 & Float >10%):")
        for _, row in squeeze_risk.iterrows():
            dtc_change = "청산 중" if row['dp_short_change_pct'] < -5 else "유지" if abs(row['dp_short_change_pct']) < 5 else "증가 중"
            print(f"   • {row['ticker']}: DTC {row['yf_short_ratio_days']:.2f}일, Float {row['yf_short_percent_float']:.2f}% - {dtc_change}")

    # 2. 청산 진행 중
    squeeze_starting = df_main[(df_main['yf_short_ratio_days'] > 5) & (df_main['dp_short_change_pct'] < -5)]
    if not squeeze_starting.empty:
        print(f"\n🟢 청산 시작 종목 (DTC >5일 & DP Short 급락):")
        for _, row in squeeze_starting.iterrows():
            finra_status = "청산 진행" if row['finra_yf_short_ratio'] < 10 else "정상"
            print(f"   • {row['ticker']}: {row['dp_short_change_pct']:+.2f}%p 급락, FINRA/YF {row['finra_yf_short_ratio']:.1f}% ({finra_status})")

    # 3. 신규 공매도 공격
    new_short_attack = df_main[(df_main['dp_short_change_pct'] > 5) & (df_main['finra_yf_short_ratio'] > 50)]
    if not new_short_attack.empty:
        print(f"\n🔴 신규 공매도 공격 진행 (DP Short 급등 & FINRA/YF >50%):")
        for _, row in new_short_attack.iterrows():
            print(f"   • {row['ticker']}: {row['dp_short_change_pct']:+.2f}%p 급등, FINRA/YF {row['finra_yf_short_ratio']:.1f}%")

    # 4. 건강한 종목
    healthy = df_main[(df_main['yf_short_ratio_days'] < 3) & (df_main['yf_short_percent_float'] < 5)]
    if not healthy.empty:
        print(f"\n✅ 건강한 종목 (DTC <3일 & Float <5%):")
        for _, row in healthy.iterrows():
            print(f"   • {row['ticker']}: DTC {row['yf_short_ratio_days']:.2f}일, Float {row['yf_short_percent_float']:.2f}%")

    # 5. 기관 매집 의심
    accumulation = df_main[(df_main['dp_ratio'] > 50) & (df_main['dp_short_ratio'] < 45)]
    if not accumulation.empty:
        print(f"\n💚 기관 매집 가능성 (DP >50% & DP Short <45%):")
        for _, row in accumulation.iterrows():
            print(f"   • {row['ticker']}: DP {row['dp_ratio']:.1f}%, DP Short {row['dp_short_ratio']:.1f}%")

    print("\n" + "="*120)
    print("📊 데이터 출처:")
    print("   • Yahoo Finance: Days to Cover, Short % of Float (표준 지표)")
    print("   • FINRA: Dark Pool 거래 데이터 (장외 거래 분석)")
    print(f"🕐 분석 완료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*120)

    print("\n💡 투자 시 주의사항:")
    print("""
    1. Days to Cover가 높다고 무조건 오르는 것은 아닙니다.
    2. 공매도 세력이 맞을 수도 있으며, 주가는 계속 하락할 수 있습니다.
    3. 하지만 일단 반등이 시작되면, DTC가 높을수록 Short Squeeze로 폭등할 가능성이 큽니다.
    4. 이 분석은 기술적 지표일 뿐, 펀더멘털 분석과 병행해야 합니다.
    5. 투자 결정은 본인의 책임이며, 이 분석은 참고 자료일 뿐입니다.
    """)

🚀 MAG 7+2: Dark Pool & Short Interest 심층 분석 (표준 지표 적용)

데이터 수집 중... AAPL.. MSFT.. GOOGL.. AMZN.. NVDA.. META.. TSLA.. COIN.. IBIT.. 
완료!

📚 핵심 지표 체계 및 상세 해석 가이드

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 📊 표준 공매도 지표 (Yahoo Finance 제공) - 가장 중요!                                          │
    └────────────────────────────────────────────────────────────────────────────────────────────────┘

    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
    │ 1️⃣ Short Ratio (Days to Cover) ⭐⭐⭐ 최우선 핵심 지표!                                              │
    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
    │ 📐 계산식: 공매도 잔고 / 평균 일일 거래량                                                      │
    │                                                                                                │
    │ 💡 의미:                                   


📊 Chart 2: Short % of Float - 유통주식 대비 공매도 비율

    💡 Short % of Float 해석:

    🔍 의미:
    • 시장에서 거래 가능한 주식 중 몇 %가 공매도되었는가?
    • 시장 참여자들의 약세 심리 강도 측정

    📊 해석 기준:
    • <2%: 매우 낮음 (시장의 강한 신뢰)
    • 2-5%: 정상 범위 (건강한 시장)
    • 5-10%: 주의 (공매도 세력 관심 증가)
    • >10%: 높음 (강한 약세 베팅, Squeeze 잠재력도 높음)

    🎯 Days to Cover와 함께 보기:
    • Float >10% + DTC >5일 = 💥 극도로 위험(또는 기회!)
    • Float <5% + DTC <3일 = ✅ 안정적 종목
    



📊 Chart 2-1: DP Ratio - Dark Pool 거래 비중

    💡 DP Ratio (Dark Pool 비중) 해석:

    🔍 의미:
    • 전체 시장 거래량 중 장외(Dark Pool)에서 거래된 비중
    • 기관 투자자들의 시장 개입 강도 측정
    • 높을수록 기관들이 '은밀하게' 거래 중

    📊 해석 기준:
    • <40%: 정상 범위 (일반적인 시장 거래)
    • 40-50%: 기관 관심 증가 (모니터링 필요)
    • >50%: 기관 과열 (강한 기관 개입)
    • >60%: 극도 과열 (비정상적 수준)

    🎯 실전 활용:
    • DP Ratio >50% + DP Short >55% = 기관 강한 약세
    • DP Ratio >50% + DP Short <45% = 기관 매집 가능성
    • DP Ratio <40% = 정상 시장, 기관 개입 낮음
    



📊 Chart 3: 공매도 종합 비교 - Dark Pool vs 전체 시장

    💡 4가지 지표 종합 비교:

    1️⃣ DP Internal Short (파란색)
       • 장외 거래 중 공매도 비율
       • 기관의 은밀한 심리

    2️⃣ DP Market Impact (회색)
       • 전체 시장 대비 장외 공매도 영향
       • 절대적 규모

    3️⃣ FINRA/YF Ratio (보라색) ⭐
       • 오늘 발생한 공매도 / 전체 잔고
       • 공매도 "신선도" 측정
       • >50% = 신규 공매도 활발
       • <10% = 청산 진행

    📈 종합 해석:
    • DP Internal 높음 + FINRA/YF 높음 = 신규 공매도 공격
    • DP Internal 높음 + FINRA/YF 낮음 = 청산 시작 (기회!)
    



📊 Chart 4: Days to Cover vs Short % Float - 위험도 매트릭스

    💡 2D 위험도 분석:

    X축: Short % of Float (공매도 비율)
    Y축: Days to Cover (청산 소요 일수)

    🔥 우상단 (빨강): 극도 위험!
       • Float 높음 + DTC 높음
       • Short Squeeze 폭발 가능 구간
       • 가장 변동성 높은 영역

    💚 좌하단 (초록): 안정적
       • Float 낮음 + DTC 낮음
       • 공매도 압력 거의 없음
       • 펀더멘털 투자 적합

    버블 크기 = DP 내부 공매도 비율
    버블 색상 = Days to Cover (빨간색일수록 위험)
    


...


🔍 META (Meta) - DTC: 1.41일, Float: 1.39%



🔍 TSLA (Tesla) - DTC: 0.84일, Float: 2.45%



🔍 COIN (Coinbase) - DTC: 1.65일, Float: 6.92%



🔍 IBIT (Bitcoin ETF) - DTC: 0.00일, Float: 0.00%



✨ 최종 분석 요약 - 핵심 인사이트

✅ 건강한 종목 (DTC <3일 & Float <5%):
   • AAPL: DTC 2.34일, Float 0.78%
   • MSFT: DTC 2.14일, Float 0.65%
   • GOOGL: DTC 1.92일, Float 1.07%
   • META: DTC 1.41일, Float 1.39%
   • AMZN: DTC 1.37일, Float 0.73%
   • NVDA: DTC 1.27일, Float 1.02%
   • TSLA: DTC 0.84일, Float 2.45%
   • IBIT: DTC 0.00일, Float 0.00%

💚 기관 매집 가능성 (DP >50% & DP Short <45%):
   • NVDA: DP 50.6%, DP Short 39.7%

📊 데이터 출처:
   • Yahoo Finance: Days to Cover, Short % of Float (표준 지표)
   • FINRA: Dark Pool 거래 데이터 (장외 거래 분석)
🕐 분석 완료: 2025-11-30 13:45:01

💡 투자 시 주의사항:

    1. Days to Cover가 높다고 무조건 오르는 것은 아닙니다.
    2. 공매도 세력이 맞을 수도 있으며, 주가는 계속 하락할 수 있습니다.
    3. 하지만 일단 반등이 시작되면, DTC가 높을수록 Short Squeeze로 폭등할 가능성이 큽니다.
    4. 이 분석은 기술적 지표일 뿐, 펀더멘털 분석과 병행해야 합니다.
    5. 투자 결정은 본인의 책임이며, 이 분석은 참고 자료일 뿐입니다.
    
